In [16]:
import pandas as pd
import altair as alt

In [48]:
#path='https://luftdaten.berlin.de/station/mc010.csv?group=pollution&period=1h&timespan=custom&start%5Bdate%5D=01.01.2020&start%5Bhour%5D=12&end%5Bdate%5D=15.07.2020&end%5Bhour%5D=12'

In [2]:
standort_daten=pd.read_excel(r"blume_alle16.xlsx", sheet_name='Tabelle1', nrows=16)

standort_daten=standort_daten[['ID', 'ID2', 'Name', 'station_start_date', 'type_of_station',
       'station_type_of_area', 'station_latitude_d', 'station_longitude_d']]
standort_daten

,ID,ID2,Name,station_start_date,type_of_station,station_type_of_area,station_latitude_d,station_longitude_d
0,mc010,DEBE010,010 Wedding,19841101,background,urban,52.543041,13.349326
1,mc018,DEBE018,018 Schöneberg,19861101,background,urban,52.485814,13.348775
2,mc027,DEBE027,027 Marienfelde,19890401,background,rural,52.398406,13.368103
3,mc032,DEBE032,032 Grunewald,19861001,background,rural,52.473192,13.225144
4,mc042,DEBE034,042 Neukölln,19860301,background,urban,52.489451,13.430844
5,mc077,DEBE051,077 Buch,20170724,background,suburban,52.644169,13.483079
6,mc085,DEBE056,085 Friedrichshagen,19940201,background,rural,52.447697,13.647050
7,mc145,DEBE062,145 Frohnau,19960201,background,rural,52.653269,13.296081
8,mc171,DEBE068,171 Mitte,20030101,background,urban,52.513606,13.418833
9,mc282,DEBE066,282 Karlshorst,19990701,background,urban,52.485296,13.529504


# downlaod raw data via API

In [10]:
import csv
import requests
import os

stationen=list(standort_daten.ID) #station='mc220'
jahre= list(range(2012,2021))#jahr='2018'

for station in stationen:
    
    #newfolder for each station
    if not os.path.exists(station):
        os.makedirs(station)
        
    for jahr in jahre:
        jahr=str(jahr)
        url = 'https://luftdaten.berlin.de/station/'+station+'.csv?group=pollution&period=1h&timespan=custom&start%5Bdate%5D=01.01.'+jahr+'&start%5Bhour%5D=0&end%5Bdate%5D=31.12.'+jahr+'&end%5Bhour%5D=23'
        response = requests.get(url)
        
        with open(station+'/'+station+'_'+jahr+'.csv', 'w') as f:
            writer = csv.writer(f)
            for line in response.iter_lines():
                writer.writerow(line.decode('utf-8').split(','))

# load raw data into pandas df

In [11]:
import glob

#station_files= glob.glob(station+"_*.csv")
#station_files= glob.glob("*.csv")
station_files= glob.glob("mc*/*.csv")

print(station_files)

['mc010\\mc010_2012.csv', 'mc010\\mc010_2013.csv', 'mc010\\mc010_2014.csv', 'mc010\\mc010_2015.csv', 'mc010\\mc010_2016.csv', 'mc010\\mc010_2017.csv', 'mc010\\mc010_2018.csv', 'mc010\\mc010_2019.csv', 'mc010\\mc010_2020.csv', 'mc018\\mc018_2012.csv', 'mc018\\mc018_2013.csv', 'mc018\\mc018_2014.csv', 'mc018\\mc018_2015.csv', 'mc018\\mc018_2016.csv', 'mc018\\mc018_2017.csv', 'mc018\\mc018_2018.csv', 'mc018\\mc018_2019.csv', 'mc018\\mc018_2020.csv', 'mc027\\mc027_2012.csv', 'mc027\\mc027_2013.csv', 'mc027\\mc027_2014.csv', 'mc027\\mc027_2015.csv', 'mc027\\mc027_2016.csv', 'mc027\\mc027_2017.csv', 'mc027\\mc027_2018.csv', 'mc027\\mc027_2019.csv', 'mc027\\mc027_2020.csv', 'mc032\\mc032_2012.csv', 'mc032\\mc032_2013.csv', 'mc032\\mc032_2014.csv', 'mc032\\mc032_2015.csv', 'mc032\\mc032_2016.csv', 'mc032\\mc032_2017.csv', 'mc032\\mc032_2018.csv', 'mc032\\mc032_2019.csv', 'mc032\\mc032_2020.csv', 'mc042\\mc042_2012.csv', 'mc042\\mc042_2013.csv', 'mc042\\mc042_2014.csv', 'mc042\\mc042_2015.csv',

In [12]:
import pandas as pd
data=pd.DataFrame()

for file in station_files:
    data0=pd.read_csv(file, encoding='latin1', sep=';', skiprows=1)[3:]

    data0=data0.rename(columns={'Messkomponente':'timestamp'})
    data0['ID']=file[:5]

    data=data.append(data0)#, ignore_index=True)
data['timestamp'] =  pd.to_datetime(data['timestamp'], format='%d.%m.%Y %H:%M' )#'%Y%m%d-%H%M%S'

data

C:\Users\Simon\Anaconda3\envs\geo_julab\lib\site-packages\pandas\core\frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


,Benzol,Feinstaub (PM10),ID,Kohlenmonoxid,Ozon,Schwefeldioxid,Stickoxide,Stickstoffdioxid,Stickstoffmonoxid,Toluol,timestamp
3,1.5,NaN,mc010,NaN,10,NaN,43,34,6,2.2,2012-01-01 00:00:00
4,2.2,NaN,mc010,NaN,3,NaN,108,41,43,2.2,2012-01-01 01:00:00
5,3.1,NaN,mc010,NaN,3,NaN,63,38,17,2.6,2012-01-01 02:00:00
6,2.0,NaN,mc010,NaN,5,NaN,45,34,8,2.5,2012-01-01 03:00:00
7,1.8,NaN,mc010,NaN,4,NaN,46,31,9,3.1,2012-01-01 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...
4834,NaN,NaN,mc282,NaN,NaN,NaN,41,32,6,NaN,2020-07-20 08:00:00
4835,NaN,NaN,mc282,NaN,NaN,NaN,45,36,6,NaN,2020-07-20 09:00:00
4836,NaN,NaN,mc282,NaN,NaN,NaN,32,27,3,NaN,2020-07-20 10:00:00
4837,NaN,NaN,mc282,NaN,NaN,NaN,15,13,1,NaN,2020-07-20 11:00:00


# let the wringling beginn

In [209]:
import datetime

In [19]:
import geopandas as gpd
import topojson 

bezirke_layer=gpd.read_file(r"..\\..\\berlin_vzaehlung\\shapes\\BerlinerBezirke.geojson")

bezirke_layer=bezirke_layer[['name','geometry']].copy()
bezirke_layer=bezirke_layer[bezirke_layer.name.notnull()].copy()
bezirke_layer.geometry=bezirke_layer.buffer(0.00001)
bezirke_layer.name=bezirke_layer.name.str.replace('ö','oe')


#krs.geometry= krs.buffer(0) #falls repair geometry notwendig\n",
bezirke_layer_tj = topojson.Topology(bezirke_layer, prequantize=False, topology=True)
bezirke_layer_tj_sim = bezirke_layer_tj.toposimplify(0.002)
bezirke_layer = bezirke_layer_tj_sim.to_gdf()
bezirke_layer.crs='epsg:4326'

bezirke=alt.Chart(bezirke_layer).mark_geoshape(stroke='white',strokeWidth=0.5).encode()
bezirke

C:\Users\Simon\Anaconda3\envs\geo_julab\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  


alt.Chart(...)

In [17]:
standort_daten

,ID,ID2,Name,station_start_date,type_of_station,station_type_of_area,station_latitude_d,station_longitude_d
0,mc010,DEBE010,010 Wedding,19841101,background,urban,52.543041,13.349326
1,mc018,DEBE018,018 Schöneberg,19861101,background,urban,52.485814,13.348775
2,mc027,DEBE027,027 Marienfelde,19890401,background,rural,52.398406,13.368103
3,mc032,DEBE032,032 Grunewald,19861001,background,rural,52.473192,13.225144
4,mc042,DEBE034,042 Neukölln,19860301,background,urban,52.489451,13.430844
5,mc077,DEBE051,077 Buch,20170724,background,suburban,52.644169,13.483079
6,mc085,DEBE056,085 Friedrichshagen,19940201,background,rural,52.447697,13.647050
7,mc145,DEBE062,145 Frohnau,19960201,background,rural,52.653269,13.296081
8,mc171,DEBE068,171 Mitte,20030101,background,urban,52.513606,13.418833
9,mc282,DEBE066,282 Karlshorst,19990701,background,urban,52.485296,13.529504


In [20]:
single_nearest = alt.selection_single(on='mouseover', empty='none')

points=alt.Chart(standort_daten).mark_circle(
    color='black',
    stroke='white',strokeWidth=0.0
).encode(
    longitude='station_longitude_d:Q',
    latitude='station_latitude_d:Q',
    size=alt.condition(single_nearest, alt.value(100), alt.value(40)),
).add_selection(
    single_nearest
)

bezirke + points

alt.LayerChart(...)

In [ ]:
##########################################################

In [ ]:
alt.data_transformers.disable_max_rows()

In [26]:

######### vorbereitung

import math
def getQuartal(x):
    return math.ceil(x/3)

data['Quartal']=data.timestamp.dt.month.apply(lambda x: getQuartal(x))

data['wtag']=data.timestamp.dt.weekday.apply(lambda x: 'Wochenentag' if x>4 else 'Werktag')


data['Feinstaub (PM10)']=data['Feinstaub (PM10)'].astype(float)
data['Stickstoffdioxid']=data['Stickstoffdioxid'].astype(float)
data_grp=data.groupby(['ID',
             data.timestamp.dt.hour.rename('hour'), 
             #data.timestamp.dt.month.rename('month'),
             data['Quartal'],
             data.timestamp.dt.year.rename('year'),
             #data.timestamp.dt.weekday_name.rename('weekday')
             data['wtag']
                      ])['Feinstaub (PM10)','Stickstoffdioxid'].mean().reset_index()

#data['Quartal']=data['Quartal'].astype(int)


#source = data_grp#[data_grp.month<=6]
source=pd.melt(data_grp, id_vars=['ID','hour','Quartal','wtag','year'], value_vars=['Feinstaub (PM10)','Stickstoffdioxid'])

######### viz
#selections
selection = alt.selection_single(fields=['ID'], init={'ID': 'mc143'})
#selection_year = alt.selection_single(fields=['weekday'], bind='legend')

selection_variable = alt.selection_single(fields=['variable'], init={'variable': 'Stickstoffdioxid'})
selection_week = alt.selection_multi(fields=['wtag'], init=[{'wtag': 'Werktag'}])
#selection_month= alt.selection_multi(fields=['month'])
#selection_quarter= alt.selection_single(fields=['Quartal'], init={'Quartal': '1'})
selection_quarter= alt.selection_multi(fields=['Quartal'], init=[{'Quartal': 1}])

selection_year= alt.selection_multi(fields=['year'])

single_nearest = alt.selection_single(on='mouseover', empty='none')
single_nearest_2 = alt.selection_single(on='mouseover', empty='none',fields=['typea'])
single_nearest_3 = alt.selection_single(on='mouseover', empty='none',fields=['typeb'])
##charts





line = alt.Chart(source).mark_line(point=True).encode(
    #x='hour',
    x=alt.X('hour', scale=alt.Scale(domain=(0,23))),
    y=alt.Y('mean(value):Q', title='Schadstoffwert',scale=alt.Scale(domain=(0,100))),
    #color=alt.Color('weekday:O',scale=alt.Scale(domain=['Monday', 'Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'])),
    color=alt.Color('year:O'),
    shape=alt.Shape('Quartal:O'),
    #opacity=alt.condition(selection_year, alt.value(1), alt.value(0.2)),
    detail=['Quartal:O','year:O','wtag:O'],
    tooltip=['year:N','Quartal','wtag','variable:O','mean(value)']
   # tooltip=['weekday','mean(value))']
).transform_filter(
    selection
).transform_filter(
    selection_variable
).transform_filter(
    selection_week
).transform_filter(
    selection_quarter
).transform_filter(
    selection_year
).add_selection(
    selection_year,selection,selection_variable,selection_week,selection_quarter,selection_year
).properties(width=600, height=500
)


## interactive legend
legend_schad = alt.Chart(source.drop_duplicates('variable'), title='Schadstoff').mark_rect(stroke='1',strokeWidth=0.2).encode(
        y=alt.Y('variable:N', axis=alt.Axis(title='', orient='right')),#,title='Bundesländer'),#  sort=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]), , titleAnchor  ='start'
        color=alt.condition(selection_variable,
                    alt.value('gray'),
                    alt.value('lightgray')) , 
    
        #tooltip=['BLand:N', 'count(BLand)'],#,'mean(cnt):Q']
        #tooltip=[
        #alt.Tooltip('count(Bundesland):Q', title='Messstation mit DTV'),
        #alt.Tooltip('bland_freq', title='gesamte Messstationen in Bundesland')
        #],
    
    ).add_selection(
    selection_variable
).properties(
        width=20,
    )

## interactive legend
legend_week = alt.Chart(source.drop_duplicates('wtag'), title='Wochentag').mark_rect(stroke='1',strokeWidth=0.2).encode(
        y=alt.Y('wtag:O', axis=alt.Axis(title='', orient='right')),# scale=alt.Scale(domain=['Werktag','Wochenendtag'])),#,title='Bundesländer'),#  sort=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]), , titleAnchor  ='start'
        color=alt.condition(selection_week,
                    alt.value('gray'),
                    alt.value('lightgray')) , 
    
        #tooltip=['BLand:N', 'count(BLand)'],#,'mean(cnt):Q']
        #tooltip=[
        #alt.Tooltip('count(Bundesland):Q', title='Messstation mit DTV'),
        #alt.Tooltip('bland_freq', title='gesamte Messstationen in Bundesland')
        #],
    
    ).add_selection(
    selection_week
).properties(
        width=20,
    )

## interactive legend
legend_quarter = alt.Chart(source.drop_duplicates('Quartal'), title='Quartal').mark_rect(stroke='1',strokeWidth=0.2).encode(
        y=alt.Y('Quartal:N', axis=alt.Axis(title='', orient='right')),#,title='Bundesländer'),#  sort=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]), , titleAnchor  ='start'
        color=alt.condition(selection_quarter,
                    alt.value('gray'),
                    alt.value('lightgray')) , 
    
        #tooltip=['BLand:N', 'count(BLand)'],#,'mean(cnt):Q']
        #tooltip=[
        #alt.Tooltip('count(Bundesland):Q', title='Messstation mit DTV'),
        #alt.Tooltip('bland_freq', title='gesamte Messstationen in Bundesland')
        #],
    
    ).add_selection(
    selection_quarter
).properties(
        width=20,
    )

## interactive legend
legend_year = alt.Chart(source.drop_duplicates('year'), title='Jahr').mark_rect(stroke='1',strokeWidth=0.2).encode(
        y=alt.Y('year:N', axis=alt.Axis(title='', orient='right')),#,title='Bundesländer'),#  sort=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]), , titleAnchor  ='start'
        color=alt.condition(selection_year,
                    alt.value('gray'),
                    alt.value('lightgray')) , 
    
        #tooltip=['BLand:N', 'count(BLand)'],#,'mean(cnt):Q']
        #tooltip=[
        #alt.Tooltip('count(Bundesland):Q', title='Messstation mit DTV'),
        #alt.Tooltip('bland_freq', title='gesamte Messstationen in Bundesland')
        #],
    
    ).add_selection(
    selection_year
).properties(
        width=20,
    )


source_helper=source[:2].copy()
source_helper['line_pos_y']=40
#y='line_pos_y',
holine = alt.Chart(source_helper).mark_rule(color='red',strokeWidth=0.5).encode(
    y='line_pos_y'
)

#line2 = alt.Chart(source).mark_line(point=True).encode(
#    #x='hour',
#    x=alt.X('hour', scale=alt.Scale(domain=(0,23))),
#    y=alt.Y('mean(Stickstoffdioxid):Q', title='Stickstoffdioxid',scale=alt.Scale(domain=(0,100))),
#    color=alt.Color('weekday:O',scale=alt.Scale(domain=['Monday', 'Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'])),
#    opacity=alt.condition(selection_year, alt.value(1), alt.value(0.2)),
#    tooltip=['weekday','mean(Stickstoffdioxid)):Q']
#).transform_filter(
#    selection
#).add_selection(
#    selection_year,selection
#).properties(width=600, height=500)
#
#source_helper=source[:2].copy()
#source_helper['line_pos_y']=50
##y='line_pos_y',
#holine = alt.Chart(source_helper).mark_rule(color='red',strokeWidth=0.5).encode(
#    y='line_pos_y'
#)



#maps
bezirke=alt.Chart(bezirke_layer).mark_geoshape(stroke='white',strokeWidth=0.5, color='lightgray').encode()

#points=alt.Chart(standort_daten).transform_calculate(
#    url='https://luftdaten.berlin.de/station/'+alt.datum.ID+'#station-info'
#).mark_circle(
points=alt.Chart(standort_daten).mark_circle(
    color='black',
    stroke='white',
    strokeWidth=0.0,
    #size=600
).encode(
    longitude='station_longitude_d:Q',
    latitude='station_latitude_d:Q',
    size=alt.condition(single_nearest, alt.value(250), alt.value(150)),
   # text=alt.Text('Beschreibung - Fahrtrichtung:O')#
    color='type_of_station',
    #size=alt.value(20)
    #href='url:N',#url=''
).add_selection(
    selection,single_nearest
).properties(width=600, height=500)

text=alt.Chart(standort_daten).mark_text(
    color='black',dy=-10, strokeWidth=0
).encode(
    longitude='station_longitude_d:Q',
    latitude='station_latitude_d:Q',
    text=alt.Text('Name:O'),
    fillOpacity=alt.condition(single_nearest, alt.value(1), alt.value(0)),
)



###### TEXT SAUCE ########
y_pos=-65


#TEXT IN CHART
text_source=alt.Chart(standort_daten).transform_calculate(
    textt= alt.datum.Name 
).mark_text(
    color='black',# dx=-240, dy=-240
    align='left'
).encode(
    text=alt.Text('textt:O'),
    opacity=alt.condition(selection, alt.value(1), alt.value(0)),
    x=alt.value(30),
    y=alt.value(y_pos),
)


#TEXT IN CHART
text_source2=alt.Chart(standort_daten).transform_calculate(
    textt= "'Stations-Info'"
).mark_text(
    color='black',# dx=-240, dy=-240
    align='left'
).encode(
    text=alt.Text('textt:O'),
    opacity=alt.condition(selection, alt.value(1), alt.value(0)),
    x=alt.value(50),
    y=alt.value(y_pos+20),
)

#TEXT IN CHART
text_source3=alt.Chart(standort_daten).transform_calculate(
    textt= "'Stations-Rohdaten (csv)'"
).mark_text(
    color='black',# dx=-240, dy=-240
    align='left'
).encode(
    text=alt.Text('textt:O'),
    opacity=alt.condition(selection, alt.value(1), alt.value(0)),
    x=alt.value(50),
    y=alt.value(y_pos+40),
)


#LINK POINT
text_source_links=alt.Chart(standort_daten).transform_calculate(
    url= 'https://luftdaten.berlin.de/station/'+alt.datum.ID+ '#station-info'
).mark_circle(
    color='black',
    size=250
).encode(
    x=alt.value(40),
    y=alt.value(y_pos+20),
    #size=alt.condition(single_nearest_2, alt.value(250), alt.value(150)),
    #color=alt.condition(single_nearest_2, alt.value('black'), alt.value('gray')),
    href='url:N',#url=''
).add_selection(
    single_nearest_2
).transform_filter(
    selection
)

#single_nearest_3 = alt.selection_single(on='mouseover', empty='none')
#LINK POINT
text_source_links2=alt.Chart(standort_daten).transform_calculate(
    url= 'https://github.com/vizsim/luftdaten_berlin/data/'+alt.datum.ID
).mark_circle(
    color='black',
    size=250
).encode(
    x=alt.value(40),
    y=alt.value(y_pos+40),
    #size=alt.condition(single_nearest_3, alt.value(250), alt.value(150)),
    #color=alt.condition(single_nearest_3, alt.value('black'), alt.value('gray')),
    href='url:N',#url=''
).add_selection(
    single_nearest_3
).transform_filter(
    selection
)


texte= text_source + text_source2 + text_source3
texte_links= text_source_links + text_source_links2

#final_chart=alt.hconcat(
#    legend,
#    (texte_links+line+holine+texte).resolve_legend(color="independent").resolve_scale(x='shared',color="independent"),
#    #line2,
#    (bezirke + points+ text)
#).configure_view(stroke=None)
#
#
#
#
#
##final_chart.save('out_both_weekday.html')
#final_chart


legends=(legend_schad & legend_week & legend_quarter & legend_year)
line_chart= (texte_links+line+holine+texte).resolve_legend(color="independent",shape="independent").resolve_scale(x='shared',color="independent")
map_chart=  (bezirke + points+ text).resolve_legend(color="independent").resolve_scale(x='shared',color="independent") 

plot=legends | line_chart |  map_chart


# adding Titel and Subtitle
title = alt.Chart(
    {"values": [{"text": "Durchschnittl. stündliche Luftbelastung je Wochentag/Quartal/Jahr in Berlin (2012-2020)"}]}
).mark_text(size=20, align='left').encode(  #,anchor='start' , align='left'
    text="text:N",
    x=alt.value(0)
)

subtitle = alt.Chart(
    {"values": [{"text": 'inkl. Link zu Stationsinfo und Downloadmöglichkeit der Rohdaten'}]}
).mark_text(size=11, align='left').encode(
    text="text:N",
    x=alt.value(0)
)
subtitle2 = alt.Chart(
    {"values": [{"text": 'Mehrfachauswahl mit gedrückter SHIFT-Taste möglich.'}]}
).mark_text(size=11, align='left').encode(
    text="text:N",
    x=alt.value(0)
)

newline_helper = alt.Chart(
    {"values": [{"text": '  '}]}
).mark_text(size=11, align='left').encode(
    text="text:N",
    x=alt.value(0)
)

quelle = alt.Chart(
    {"values": [{"text": 'Quelle: https://luftdaten.berlin.de'}]}
).mark_text(size=11, align='left').encode(
    text="text:N",
    x=alt.value(0),
    y=alt.value(40)
    #yOffset=10
)
  
fullchart_title= alt.vconcat(
    title,
    subtitle,subtitle2,newline_helper,#
    plot,
    quelle
).configure_view(
    stroke=None
).configure_concat(
    spacing=1
).configure_point(
    size=70
)

fullchart_title.save('out_both_weekday_v05.html')
#fullchart_title